# Time dependent Diffusion
In this note, we consider a diffusion problem, which is the simplest extension of the Poisson problem into the time domain. The solution $u= u(t,x,y)$, with initial $u_0(t, x, y)$ and boundary $I(x,y)$.
\begin{equation}
\begin{aligned}
\frac{\partial u}{\partial t} & =\Delta u+f \text { in } \Omega, \text { for } t>0 \\
u & =u_0 \text { on } \partial \Omega, \text { for } t>0 \\
u & =I \text { at } t=0 .
\end{aligned}
\end{equation}